# COF_plot_stat.py

O seguinte programa faz a plotagem e tratamento estatístico dos arquivos .csv obtidos nos ensaios *reciprocating* em Tribômetro Bruker UMT. Após o ensaio, o arquivo .tst é interpretado pelo programa Viewer e exportado para .csv, o qual é lido pelo programa abaixo.  

## Bibliotecas

O programa utiliza as bibliotecas:  
    - Numpy: para trabalhar com operações e constantes matemáticas  
    - Pandas: para trabalhar com DataFrames e DataSeries, que são formas de armazenar os dados do ensaio. A primeira é multi-dimensional, enquanto a segunda é unidimensional  
    - Matplotlib: para plotar os gráficos  
    - Axes3D: para gráficos 3D  
    - Matplotlib.cm: para esquemas de cores  

In [1]:
#bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cmx

## Funções
As funções são utilizadas para deixar o programa mais limpo e de mais fácil leitura. São elas:
- scatter3d(): para plotar um gráfico de dispersão tridimensional, onde um dos eixos tem codificação de cor
- OpenCSVs(): abre os arquivos .csv e armazena em uma lista (data) em que cada elemento é um DataFrame de um arquivo
- getDuration(): retorna a duração de um ensaio passado como parâmetro na forma de DataFrame
- getFrequency(): retorna a frequência de oscilação utilizada no ensaio. Assume que a duração do ensaio é função apenas da frequência.
- getZero(): retorna o tempo em segundos que aparece o mínimo coeficiente de atrito, ou seja, o tempo onde a velocidade é revertida.
- getVelocity(): retorna a velocidade instantânea no MHS, com base nos parâmetros passados

### scatter3d()
**Entradas**
- x: coluna de dados de um DataFrame correspondente ao eixo x do gráfico 3D
- y: coluna de dados de um DataFrame correspondente ao eixo y do gráfico 3D
- z: coluna de dados de um DataFrame correspondente ao eixo z do gráfico 3D
- cs: coluna de dados de um DataFrame utilizada para determinar as cores dos pontos
- it: número a ser colocado no arquivo de saída .png
- colorsMap: esquema de cores utilizado; padrão 'coolwarm' azul(-) e vermelho(+)

**Saídas**
- retorna: nada
- salva o arquivo .png em disco

**Configurações**
- ax.set_(x/y/z)label('string'): Muda o rótulo do eixo x/y/z para 'string'
- ax.set_(x/y/z)lim(a,b): configura os pontos zero e máximo do eixo x/y/z para a e b, respectivamente
- ax.(x/y/z)axis.labelpad=N: configura um espaçamento em torno dos rótulos dos eixos (x/y/z) igual a N
- ax.dist = M: configura uma distância de 'observação' do gráfico, necessário para não cortar as legendas
- shrink = 0.8 (em fig.colorbar): configura o redimensionamento da barra de cores em relação a altura da figura
- local e material: variaveis globais (provavelmente seja melhor alterar para parâmetros)

In [3]:
#funções
def scatter3d(x,y,z, cs,it, colorsMap='coolwarm'):
    cm = plt.get_cmap(colorsMap)
    cNorm = cmx.colors.Normalize(vmin=min(cs), vmax=max(cs))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x, y, z, c=scalarMap.to_rgba(cs),s=0.1,)
    ax.set_zlabel('COF')
    ax.set_zlim(0,0.5)
    ax.set_ylabel('Time')
    ax.set_xlabel('Position')
    ax.xaxis.labelpad=20
    ax.yaxis.labelpad=20
    ax.zaxis.labelpad=20
    ax.dist = 13
    scalarMap.set_array(cs)
    fig.colorbar(scalarMap, shrink = 0.8)
    plt.savefig(local+'/'+material+'/'+'Graphs'+'/'+'Triboscopia_'+str(it),dpi=300,)

### OpenCSVs()
**Entradas**
- local: string com o local do arquivo (pasta com todos os CSVs). A notação deve ser com barras /, não com contrabarras \
- material: string com o nome da pasta com o material dentro de local
- nome: string com o nome do arquivo (exceto _###)
- inicio: int, primeiro número de arquivo
- fim: int, último número de arquivo
- colunas: quais colunas importar do arquivo do tribômetro

**Saídas**
- dados: lista retornada. Cada elemento da lista é um DataFrame com todos os dados de cada experimento. Caso um arquivo não seja encontrado, o elemento terá valor None

In [ ]:
def OpenCSVs(local,material,nome,inicio,fim,colunas):
    """
    abre os arquivos .csv na string 'local', string 'nome', desde o índice de
    arquivo _00 inicio até _ fim. Somente importa as colunas passadas na lista
    de inteiros colunas
    """
    dados = []
    for i in range(fim-inicio+1):
        if i<=8:
            filename = local+'/'+material+'/'+nome+'_00'+str(i+1)+'.csv'
        elif i<=98:
            filename = local+'/'+material+'/'+nome+'_0'+str(i+1)+'.csv'
        else:
            filename = local+'/'+material+'/'+nome+'_'+str(i+1)+'.csv'
        try:
            dados.append(pd.read_csv(filename, header=17, skiprows = [21], 
                        usecols = colunas,
                        dtype = {'Fx': np.float64, 'Fz': np.float64,
                                 'COF': np.float64}))
        except FileNotFoundError:
            data.append(None)
            continue
    return dados

### getDuration()
**Entradas**
- ensaio: DataFrame com um ensaio

**Saídas**
- duracao: retorna um int com o número de milissegundos (assumindo taxa de aquisição de 1000Hz) de duracao

**OBS**
Não está sendo utilizado, substituido por DataFrame['T'].max()

In [ ]:
def getDuration(ensaio):
    duracao = ensaio.shape[0]
    return duracao

### getFrequency()
**Entradas**
- ensaio: DataFrame com um ensaio

**Saídas**
- freq: inteiro com valor 2 ou 4, dependendo da duração do ensaio *(funciona apenas para distância constante)*. Caso não consiga encontrar um valor, a frequência retorna 0.

In [ ]:
def getFrequency(ensaio):  
    if ensaio.index.max() >= 1615000 and ensaio.index.max() <= 1616000:
        freq = 2.
    elif ensaio.index.max() >= 809000 and ensaio.index.max() <= 810000:
        freq = 4.
    else:
        freq = 0
    return freq

### getZero()
**Entradas**
- ensaio: DataFrame com um ensaio

**Saídas**
- retorna o valor de 'T' (coluna 0) cujo COF seja o menor, considerando os últimos 100 segundos de ensaio

In [ ]:
def getZero(ensaio):
    return ensaio.tail(n = 100000).nsmallest(1,'COF').iat[0,0]

### getVelocity()
**Entradas**
- t: tempo no instante que se deseja calcular a velocidade
- t0: tempo no qual o coeficiente de atrito é mínimo (quando a velocidade reverte)
- stroke: comprimento do stroke no ensaio
- f: frequencia

**Saídas**
- V: float, retorna a velocidade instantânea em mm/s

In [ ]:
def getVelocity(t,t0,stroke,f):
    V = (- stroke * np.pi * f * np.sin(2.*np.pi*f*((t-t0))))
    return V

## Programa Principal
### Dados de Entrada
- startfile: número do primeiro arquivo a ser lido (int)
- endfile: número do último arquivo a ser lido (int)
- local: caminho para a pasta onde estão os arquivos .csv (string)
- material: nome da pasta onde estão os arquivos de determinado material (string)
- name: nome do arquivo (sem a parte \_números) (string)
- columns: colunas que devem ser importadas do .csv (lista de ints)

In [ ]:
#DADOS ENTRADA
startfile = 1
endfile = 12
local = 'C:/Users/caiot/OneDrive/Documentos/Python/Results/CSV'
material = 'PUEG2'
name = 'PV_Limite'
columns = [0,1,2,3,6]

### Dados de Ensaio
- distancia: em metros, distancia total de deslizamento
- stroke: em milímetros, distância de um semi-ciclo (amplitude MHS)

In [ ]:
#DADOS ENSAIO
distancia = 30
stroke = 4.65

### Dados da Análise
- L: número inteiro, onde 1/L*100 representa a percentagem final do ensaio que será analisada (definição de steady-state)

In [ ]:
#DADOS ANÁLISE
L = 20 #região analisada (últimos [1/L]*100% do ensaio)

### Passo 1
Carrega os dados brutos na lista data (cada elemento é um DataFrame de dados brutos importados do .csv).

In [ ]:
#PROGRAMA PRINCIPAL
#%%
#Lista data de dados brutos
print('Carregando dados...')
data = OpenCSVs(local,material,name,startfile,endfile,columns)
print('Dados carregados com sucesso!')

### Passo 2
Seleciona os dados da percentagem final (tempo/tempo) do ensaio. Os elementos da lista **end** são dataframes com os dados da porção final dos ensaios.

In [ ]:
#%%
#Lista end de últimos 100/L% segundos de ensaio
end = []
print('Selecionando dados de fim de ensaio (steady-state)...')
for ensaio in data:
    end.append(ensaio.tail(n = 1 + ensaio.shape[0]//L))
print('Dados selecionados com sucesso! Total de: ' + str(len(end)) + ' ensaios.')

### Passo 3
Os elementos da lista **newend** contém duas colunas a mais que **end**:
- X: Com as posições em relação ao MHS
- V: Com as velocidades instantâneas a cada ponto

In [ ]:
#%%
#Lista newend = end + coluna(X) + coluna(V)
newend = []
for df in end:
    f = getFrequency(df)
    zero = getZero(df)
    df = df.assign(V = - stroke * np.pi * f * 
                       np.sin((2.*np.pi*f)*(df['T'] - zero)),
                    X = stroke * np.cos(2.*np.pi*f*(df['T']-zero)))
    newend.append(df)

### Passo 4
Os elementos da lista **newdata** contêm duas colunas a mais que **data**:
- X: Com as posições em relação ao MHS
- V: Com as velocidades instantâneas a cada ponto

OBS: Fazer este passo antes do Passo 2 e selecionando os pontos finais para a **end** por meio da **newdata** pode acelerar o programa.

In [ ]:
#%%
#Lista newdata = data + coluna(X) + coluna(V)
newdata = []
for df in data:
    f = getFrequency(df)
    zero = getZero(df)
    df = df.assign(V = - stroke * np.pi * f * 
                       np.sin((2.*np.pi*f)*(df['T'] - zero)),
                    X = stroke * np.cos(2.*np.pi*f*(df['T']-zero)))
    newdata.append(df)

### Passo 5
Listas **end_v(1/2)** são criadas, para separar o COF numa direção (e acima da velocidade média) da outra direção. 

In [ ]:
#%%
# Lista end_v apenas com pontos onde V > Vmed
print('Selecionando pontos com velocidade superior da média...')
end_v1 = []
end_v2 = []
i = 0
for df in newend:
    i = i + 1
    Vmed = (distancia * 1000) / df['T'].max()
    print('... para o ensaio '+str(i)+'...')
    end_v1.append(df[df.V >= Vmed])
    end_v2.append(df[df.V <= -Vmed])
print('Processo de seleção finalizado com sucesso!')

### Passo 6
As estatísticas são calculadas para as listas **end_v1** e **end_v2**. Então uma lista de estatísticas é gerada a partir das maiores observações de COF_médio para cada lista. Ou seja, está sendo considerado o coeficiente de atrito maior em relação as direções de deslizamento. O motivo para a discrepância de COFs para as direções pode ser a falta de nivelamento da amostra em relação ao movimento.

In [ ]:
#%%
print('Calculando estatísticas...')
stat1 = []
stat2 = []
stat = []
dados_max = []
for ensaio in end_v1:
    stat1.append(ensaio.describe())
for ensaio in end_v2:
    stat2.append(ensaio.describe())
for index in range(endfile-startfile+1):
    if (stat1[index].at['mean','COF']>=stat2[index].at['mean','COF']):
        stat.append(stat1[index])
        dados_max.append(1)
    else:
        stat.append(stat2[index])
        dados_max.append(2)
print('Estatísticas calculadas com sucesso!')

### Passo 7
As estatísticas calculadas no passo anterior são gravadas em arquivo .csv, na mesma pasta do material, em uma sub-pasta de nome 'Stats'. Apenas o COF_médio e COF_desvpad são exportados.

In [ ]:
#%%    
print('Gerando saída das estatísticas...')
csv_out = pd.DataFrame()
for result in stat:
    output = pd.DataFrame([[result.at['mean','COF'],result.at['std','COF']]],
                          columns = ['mean_COF','COF_stdev'])
    csv_out = csv_out.append(output, ignore_index = True)

csv_out.to_csv(local+'/'+material+'/'+'Stats'+'/'+'stats_'+name+'.csv')
print('As estatísticas foram exportadas para o arquivo ' + name + '.csv')

### Passo 8
Plotagem dos gráficos 2D, a partir das listas **newend**, **end_v1** e **end_v2**

In [ ]:
#%%
print('Plotando gráficos 2D...')
i = 0
for ensaio in newend:          
    i = i+1
    ensaio.plot(kind='scatter',x='T',y='COF',s=0.1,ylim=[0,0.5],color='red')
    plt.savefig(local+'/'+material+'/'+'Graphs'+'/'+'COF_'+str(i),dpi=300,)
    
i = 0
for ensaio in end_v1:
    i = i+1
    if dados_max[i-1]==1:
        n = 'COF_Vmed+'
    else:
        n = 'COF_Vmed-'
    ensaio.plot(kind='scatter',x='T',y='COF',s=0.1,ylim=[0,0.5],color='blue')
    plt.savefig(local+'/'+material+'/'+'Graphs'+'/'+n+str(i),dpi=300,)
i = 0
for ensaio in end_v2:
    i = i+1
    if dados_max[i-1]==2:
        n = 'COF_Vmed+'
    else:
        n = 'COF_Vmed-'
    ensaio.plot(kind='scatter',x='T',y='COF',s=0.1,ylim=[0,0.5],color='blue')
    plt.savefig(local+'/'+material+'/'+'Graphs'+'/'+n+str(i),dpi=300,)
print('Os gráficos foram plotados com sucesso!')

### Passo 9
Plotagem dos gráficos 3D, a partir de **newdata**

In [ ]:
#%%
print('Plotando gráficos 3D...')
i = 0
for ensaio in newdata:
    i=i+1
    scatter3d(ensaio['X'],ensaio['T'],ensaio['COF'],ensaio['COF'],i)
print('Os gráficos foram plotados com sucesso!')